# 必須課題
> コーパスを選択し、ベクトル空間モデルに基づいた検索を実現せよ.
> 3種類以上のクエリでの検索結果を示すこと.

## おさらい
+ コーパスとは「文書集合」のこと
+ 辞書とは「単語」と「単語のID」との対応付けのこと
+ TF-IDFはコーパス内の各文書での単語の出現頻度に基づいた尺度の一つ

## 基本方針
1. コーパスの読み込み
2. 単語→単語IDの辞書を作成する
  + 精度が芳しくなければ、日本語のストップワードリストを利用する
3. 単語IDごとに出現回数をカウントする
4. 出現頻度に応じてTF-IDFで重み付けされた特徴ベクトルを作成する
5. 各文書のベクトル表現を作成する
6. クエリをベクトル表現に変換し、類似度の高いものを取得する

In [1]:
import json
import numpy as np
import gensim
import pandas as pd
from scipy.spatial.distance import cosine

In [2]:
with open("../data/kyoto_results_100.json", "r") as f:
    docs = json.load(f)

In [3]:
# [START 検索ロジックの基本クラス]
class BasicEngine(object):
    def __init__(self, docs):
        self.docs = docs
        self.dictionary = gensim.corpora.Dictionary([doc["bow"].split() for doc in docs])
        self.DICT_SIZE = len(self.dictionary)
        
    def _calc_cossim(self, v, w):
        return 1.0 - cosine(v, w)

In [4]:
# [START TF-IDFによる検索ロジックのクラス実装]
class TfidfEngine(BasicEngine):
    def __init__(self, docs):
        super().__init__(docs)
        id_corpus = [self.dictionary.doc2bow(doc["bow"].split()) for doc in docs]
        self.tfidf_model = gensim.models.TfidfModel(id_corpus, normalize=False)
        tfidf_corpus = self.tfidf_model[id_corpus]
        self.tfidf_vectors = gensim.matutils.corpus2dense(tfidf_corpus, self.DICT_SIZE).T

    def _calc_scores(self, query):
        tfidf_q = self.tfidf_model[self.dictionary.doc2bow(query)]
        query_vec = gensim.matutils.corpus2dense([tfidf_q], self.DICT_SIZE).T[0]
        return [self._calc_cossim(query_vec, doc_vec) for doc_vec in self.tfidf_vectors]
    
    def search(self, query, n_results=5):
        """検索結果を返す
        
        returns
        ---------
        result : list of tuple, tuple = (doc_idx, cossim, doc_title, doc_summary)
        """
        assert type(query) == set, "クエリはset型で返す"
        scores = self._calc_scores(query)
        s = pd.Series(scores)
        return [
                (idx, score, self.docs[idx]["title"], self.docs[idx]["summary"])
                for idx, score in s.sort_values(ascending=False)[:n_results].iteritems()
                ]

In [5]:
# [START 3種類のクエリで検索結果を示す]
q1 = {"京都", "紅葉"}
q2 = {"銀閣寺", "場所"}
q3 = {"鴨川", "料理"}

tfidf_engine = TfidfEngine(docs)
res1 = tfidf_engine.search(q1, n_results=10)
res2 = tfidf_engine.search(q2, n_results=10)
res3 = tfidf_engine.search(q3, n_results=10)

In [6]:
res1 # 関連度は順に L2, L1, L1, L2, L2, L2, L1, L1, L1, L1 と評価した

[(33,
  0.40981248021125793,
  '京都観光 おすすめ観光スポット・世界遺産・グルメのまとめ',
  '京都観光を楽しむ まとめ情報。京都の世界遺産や名所を紹介しています。 ... 京都観光 紅葉 京都紅葉の名称 京都紅葉おすすめ鑑賞スポット 東山や嵐山など、京都の紅葉は名所が広範囲にたくさんあるので'),
 (14,
  0.17145460844039917,
  '京都の観光タクシーは雅へ－50％割引あり！京都旅行・紅葉 ...',
  '京都の観光タクシーは50％割引価格の雅京都観光タクシー。格安貸切りタクシーで旅行スポット[紅葉・平安京や町屋・神社・舞妓など]歴史と情緒ある古都をご案内致します。'),
 (25,
  0.10897298157215118,
  '京都のお寺・神社観光コース一例／京都の観光旅行・お寺巡り ...',
  '京都の観光・お寺巡りは雅京都観光タクシー。平安京や町屋・神社・舞妓・紅葉など歴史と情緒ある古都の旅行を貸切りタクシーでご案内致します。'),
 (64,
  0.10580714792013168,
  '京都嵐山観光おすすめ！人気散策コースから周辺グルメまでご ...',
  '京都嵐山観光おすすめ！人気散策コースから周辺グルメまでご紹介！観光 京都の代表的な観光地で桜や紅葉の名所にもなっている嵐山。一日過ごしても飽きないくらい観たり食べたり買い物したりする人気スポットがいっぱいです。'),
 (2,
  0.10500672459602356,
  '京都“府”観光ガイド ～京都府観光連盟公式サイト～',
  '京都の観光情報が満載の京都府観光連盟公式サイト。スポットやイベントをエリア・ジャンルごとに検索できます。紅葉や桜の開花情報など季節のおすすめから温泉や美術館など幅広く掲載。'),
 (77,
  0.1018613874912262,
  '季節の散策コース｜観光ガイド｜そうだ 京都、行こう。～京都 ...',
  'JR東海の京都観光情報【そうだ 京都、行こう。公式サイト】。京都旅行の便利帳、散策コース紹介ページ。京都の風景、お寺や神社、桜、紅葉、イベント、お祭り、お食事やお土産、宿泊などの情報をご案内。京都が楽しくなる特集 ...'),
 (17,
  0.0970027446

In [7]:
res2 # 関連度は順に L0, L1, L1, L2, L0, L0, L0, L0, L0, L0 と評価した

[(6,
  0.14298021793365479,
  '京都パーフェクトガイド・京都観光.com',
  '本音で語る京都観光案内。おすすめの散策コース、地図、お土産、珍しいお守り、食事からやカフェ案内まで、穴場も定番もいい所いい場所を紹介します。'),
 (17,
  0.12276927381753922,
  '京都観光おすすめ50選',
  '京都観光の楽しみ方 京都観光といえば、金閣寺・銀閣寺の室町時代の将軍が建てた寺や 清水寺、高台寺、八坂神社などの東山エリアのお寺めぐりを楽しんだり 春には鮮やかな桜の花見、秋には美しい紅葉を鑑賞する京都観光や'),
 (71,
  0.11761783808469772,
  '【交通手段】金閣寺から観光名所・人気スポットへのアクセス ...',
  '金閣寺は京都を代表する観光地のひとつ。銀閣寺・嵐山・清水寺などとならび、五本の指に入る人気スポットです。ところが京都観光は移動が大変。交通手段も複雑です。そこで今回は、金閣寺から各観光名所・人気スポットへの ...'),
 (65,
  0.10005922615528107,
  '【京都観光の人気スポット】絶対に見ておくべき神社・仏閣17選！',
  '「南禅寺」は、1291年の創建以来、歴代朝廷の勅願所（※天皇の命令により、国の安全や健康の祈りなどを祈願したお寺）だった場所。京都観光の名所として高い人気を誇り、 特に桜と紅葉のシーズンに突入すると大勢の人で賑わいます。'),
 (69,
  0.095830641686916351,
  '【京都】天橋立の魂を揺さぶる人気＆おすすめ観光スポット24選 ...',
  '【京都】天橋立の魂を揺さぶる人気＆おすすめ観光スポット24選！ 京都府宮津市にある天橋立は日本三景のひとつとして人気の観光地です。アオリイカをはじめとする美味しい海産物があり、ランチをする場所にも困りません。'),
 (20,
  0.083423025906085968,
  '春夏の京都観光バスツアー予約 日帰り｜JTB 定期観光バス ...',
  '春夏の京都観光スポットを巡るおすすめバスツアー予約。バスガイドが京都のお寺や名所を案内する定期観光バスで行くJTBの日帰りバスツアー！比叡山や大原三千院、金閣寺、銀閣寺など京都観光に迷ったら

In [8]:
res3 # 関連度は順に L0, L0, L1, L1, L1, L0, L0, L0, L1, L0 と評価した

[(62,
  0.14296972751617432,
  '【京都】二条・烏丸・河原町周辺で人気の観光スポット10選 ...',
  '京都の観光スポットとして人気が高い、河原町～二条エリア。ショッピングエリアとして若者が集まる河原町をはじめ、おいしいものが集まる錦市場、川床が楽しめることでも有名な鴨川、神泉苑など平安京のころから続く名所も ...'),
 (15,
  0.084323465824127197,
  '京都定期観光バス',
  'JR京都駅前から毎日出発の京都定期観光バスの予約サイトです。 ... ワンランク上のプレミアムシリーズ 車内では2人がけシートにおひとりでゆったりお座りいただけます。9月は夜の川床料理、延暦寺特別拝観コースを、10月以降は西 ...'),
 (32,
  0.083284936845302582,
  '京都旅行｜新幹線で行くお得な京都ツアーはJR東海ツアーズ',
  '多数のアイテムから選べるセレクトクーポンで伝統の京料理、おばんざいや京漬物を堪能したり、タクシーやレンタカーで京都観光したり、京の文化体験もできちゃう！3,000円分のお土産券としても使えるよ！自由にチョイスして ...'),
 (36,
  0.07316899299621582,
  '京都 - Wikipedia',
  '京言葉 京の花街 京町家 京料理 京都市内の通り 京都の元学区 京都の難読地名 小京都 そうだ 京都、行こう。 京都・東山花灯路 京都・嵐山花灯路 京阪神 京阪 光源氏、源氏物語 外部リンク 京都 歴史文化データリンク by KYOTO DOTCOM'),
 (82,
  0.0,
  '【京都 観光 おすすめ】人気 定番 穴場の観光 ...- NAVER まとめ',
  '京都 祇園は、京都の中でも特に人気の高いエリアです。その魅力を人気＆定番スポットはもちろん穴場も紹介しながらお伝えしたいと思います。限られた時間の中で京都祇園を...'),
 (28,
  0.0,
  '京都観光タクシー 和（なごみ） | 格安料金でまごころ込めてご ...',
  '和（なごみ）京都観光タクシーからのご挨拶 なごみ京都観光タクシーは、京都の観光案内のベテランばかりで構成したドライバー群からなる京都観光タクシーグループです。私たちは「京都

In [9]:
# [START 演習課題その2で利用する評価用データの作成]
rels_q1 = ["L2", "L1", "L1", "L2", "L2", "L2", "L1", "L1", "L1", "L1"]
rels_q2 = "L0, L1, L1, L2, L0, L0, L0, L0, L0, L0".split(", ")
rels_q3 = "L0, L0, L1, L1, L1, L0, L0, L0, L1, L0".split(", ")

In [10]:
# [START データの書き出し]
# 関連度
pd.DataFrame({ "doc": ["d" + str(tup[0]) for tup in res1], "rel": rels_q1}).to_csv("../data/hw/q1.rel", header=None, sep=" ", index=False)
pd.DataFrame({ "doc": ["d" + str(tup[0]) for tup in res2], "rel": rels_q2}).to_csv("../data/hw/q2.rel", header=None, sep=" ", index=False)
pd.DataFrame({ "doc": ["d" + str(tup[0]) for tup in res3], "rel": rels_q3}).to_csv("../data/hw/q3.rel", header=None, sep=" ", index=False)
# 検索結果 top3
pd.Series(["d" + str(tup[0]) for tup in res1])[0:3].to_csv("../data/hw/q1.res", header=None, index=False)
pd.Series(["d" + str(tup[0]) for tup in res2])[0:3].to_csv("../data/hw/q2.res", header=None, index=False)
pd.Series(["d" + str(tup[0]) for tup in res3])[0:3].to_csv("../data/hw/q3.res", header=None, index=False)